In [2]:
from IncrementalTrainingApproach.iCaRL import iCaRLmodel

In [3]:
from models import get_feature_extractor

In [4]:
feature_extractor = get_feature_extractor('vit_lite')

In [5]:
model = iCaRLmodel(10,feature_extractor,64,10,2000,10,0.1)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
print(model)

In [13]:
import torch
imgs = torch.randn((3,3,32,32))


In [14]:
target = torch.tensor([1,2,3])

In [15]:
train_loader,test_loader = model._get_train_and_test_dataloader([0,10])

the size of train set is (5000, 32, 32, 3)
the size of train label is (5000,)
the size of test set is (2000, 32, 32, 3)
the size of test label is (2000,)


In [17]:
def get_one_hot(target, num_class):
    one_hot = torch.zeros(target.shape[0], num_class)
    one_hot = one_hot.scatter(dim=1, index=target.long().view(-1, 1), value=1.)
    return one_hot


In [22]:
new_target = get_one_hot(target,10)

In [23]:
new_target

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [24]:
output = model.model(imgs)

In [25]:
output.shape

torch.Size([3, 10])

In [26]:
from torch.nn import functional as F
return_val = F.binary_cross_entropy_with_logits(output,new_target)



In [27]:
print(return_val)

tensor(0.7341, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


In [34]:
model.numclass+=model.task_size

In [35]:
model.numclass

20

In [36]:
import copy
model.old_model =  copy.deepcopy(model.model)


In [37]:
print(model.old_model)



network(
  (feature): ViTLite(
    (tokenizer): Tokenizer(
      (conv_layers): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 512, kernel_size=(4, 4), stride=(4, 4))
          (1): Identity()
          (2): Identity()
        )
      )
      (flattener): Flatten(start_dim=2, end_dim=3)
    )
    (classifier): Transformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (blocks): ModuleList(
        (0): TransformerEncoder(
          (pre_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (self_attn): Attention(
            (qkv): Linear(in_features=512, out_features=1536, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=512, out_features=512, bias=True)
            (proj_drop): Dropout(p=0.1, inplace=False)
          )
          (linear1): Linear(in_features=512, out_features=1024, bias=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (norm1): LayerNorm((512,), eps=1e-05

In [38]:
classes = [model.numclass-model.task_size, model.numclass]



In [39]:
classes

[10, 20]

In [40]:
model.train_loader,model.test_loader = model._get_train_and_test_dataloader(classes)

the size of train set is (5000, 32, 32, 3)
the size of train label is (5000,)
the size of test set is (3000, 32, 32, 3)
the size of test label is (3000,)


In [42]:
import numpy as np

In [45]:
datas, labels = [],[]

In [46]:
target = torch.tensor([11,12,13])

In [48]:
new_target = get_one_hot(target,model.numclass)

In [49]:
new_target

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.]])

In [50]:
old_target = model.old_model(imgs)

In [54]:
model.model.Incremental_learning(model.numclass)

In [56]:
output = model.model(imgs)

In [58]:
print(output.shape)

torch.Size([3, 20])


In [61]:
old_target = torch.sigmoid(old_target)

In [63]:
old_target.shape


torch.Size([3, 10])

In [64]:
new_target.shape

torch.Size([3, 20])

In [67]:
old_task_size = old_target.shape[1]
old_task_size

10

In [73]:
new_target[...,:old_task_size] = old_target

In [75]:
new_target

tensor([[0.5646, 0.4944, 0.6143, 0.5941, 0.7327, 0.5278, 0.6995, 0.3335, 0.3580,
         0.4185, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.5656, 0.5241, 0.6247, 0.5699, 0.7120, 0.5415, 0.7165, 0.3197, 0.3470,
         0.4172, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.5724, 0.5172, 0.6248, 0.6027, 0.7295, 0.5115, 0.7042, 0.3290, 0.3311,
         0.4123, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000]], grad_fn=<CopySlices>)

In [78]:
output


tensor([[ 0.2599, -0.0225,  0.4655,  0.3810,  1.0083,  0.1112,  0.8450, -0.6922,
         -0.5841, -0.3288, -0.0520,  0.9260,  0.3527,  0.4571, -0.3031,  0.1416,
          0.0415, -0.4042,  0.3315, -1.1275],
        [ 0.2638,  0.0966,  0.5096,  0.2816,  0.9053,  0.1665,  0.9271, -0.7552,
         -0.6323, -0.3343, -0.1703,  0.9366,  0.3041,  0.3613, -0.3304,  0.2799,
          0.0303, -0.4780,  0.4891, -0.9540],
        [ 0.2918,  0.0687,  0.5101,  0.4167,  0.9923,  0.0461,  0.8672, -0.7126,
         -0.7034, -0.3544, -0.0359,  0.9769,  0.3813,  0.4452, -0.2919,  0.1361,
         -0.0561, -0.4883,  0.4845, -0.9265]], grad_fn=<AddmmBackward>)

In [82]:
from torch import nn 
cross_entropy = nn.CrossEntropyLoss()
bce = F.binary_cross_entropy_with_logits

In [84]:
test_output = torch.randn((2,10))
test_output

tensor([[ 0.0387, -0.4745, -0.3007,  1.3920,  0.1990, -0.4828, -0.9736, -0.1006,
         -0.6378,  1.0279],
        [-1.0514,  0.6900,  1.7681,  0.7924, -1.1241,  1.4034,  0.9899,  0.1120,
         -1.3201, -0.3777]])

In [86]:
target = torch.tensor([2,5])
target.shape

torch.Size([2])

In [87]:
loss_cross_entropy = cross_entropy(test_output,target)

In [88]:
loss_cross_entropy



tensor(2.2129)

In [89]:
target_one_hot = get_one_hot(target,10)
target_one_hot



tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [91]:
loss_bce = bce(test_output,target_one_hot)

loss_bce

tensor(0.7735)